In [1]:
import pandas as pd
import geopandas as gpd
from pprint import pprint
import matplotlib.pyplot as plt
import contextily as cx
import pickle
import os.path


In [2]:
gazetteerpath = "data/geonames/allCountries_cleaned.csv"
textfile = "data/aid_wiki.txt"
matchpickle = "data/pickles/world_matches.pickle"

df = pd.read_csv(
    gazetteerpath,
    delimiter=","
)
print(df.head())

/Users/nicibe/miniconda3/envs/geo871/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (10,11,12,13,14) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


   Unnamed: 0  geonameid                          name  \
0           0    2986043            Pic de Font Blanca   
1           1    2994701                      Roc Mélé   
2           2    3007683          Pic des Langounelles   
3           3    3017832          Pic de les Abelletes   
4           5    3023203  Port Vieux de la Coume d’Ose   

                      asciiname  \
0            Pic de Font Blanca   
1                      Roc Mele   
2          Pic des Langounelles   
3          Pic de les Abelletes   
4  Port Vieux de la Coume d'Ose   

                                      alternatenames  latitude  longitude  \
0                     Pic de Font Blanca,Pic du Port  42.64991    1.53335   
1                        Roc Mele,Roc Meler,Roc Mélé  42.58765    1.74028   
2                               Pic des Langounelles  42.61203    1.47364   
3  Pic de la Font-Negre,Pic de la Font-Nègre,Pic ...  42.52535    1.73343   
4  Port Vieux de Coume d'Ose,Port Vieux de Coume ...  4

In [3]:
placenames = df.name.tolist()
print(placenames[:10])

['Pic de Font Blanca', 'Roc Mélé', 'Pic des Langounelles', 'Pic de les Abelletes', 'Port Vieux de la Coume d’Ose', 'Port de la Cabanette', 'Port Dret', 'Costa de Xurius', 'Xixerella', 'Roc del Xeig']


In [4]:
with open(textfile, "r") as wiki:
    lines = wiki.readlines()
words = []
for line in lines:
    words.extend(line.strip().split(" "))

In [5]:
# if os.path.isfile(matchpickle):
#     with open(matchpickle, "rb") as af_match:
#         af_matches = pickle.load(af_match)
# else:
placenames_set = set(placenames)
af_matches = []
wordslen = len(words)

for i, word in enumerate(words):
    if word in placenames_set:
        # print("match!")
        af_matches.append(word)

    # if i % 1000 == 0:
    #     print(f"{i} of {wordslen}")
    # with open(matchpickle, "wb") as af_match:
    #     pickle.dump(af_matches, af_match)


In [9]:
d = {}
len_af_matches = len(af_matches)

for i, match in enumerate(af_matches):
    if i % 50 == 0:
        print(f"{i} of {len_af_matches}")
    if match in set(d.keys()):
        d[match]["count"] += 1
    else:
        d[match] = {
            "count": 1,
            "lat": df.loc[df['name'] == match].reset_index().at[0, "latitude"],
            "lon": df.loc[df['name'] == match].reset_index().at[0, "longitude"]
        }


0 of 927
50 of 927
100 of 927
150 of 927
200 of 927


KeyboardInterrupt: 

In [ ]:
geo = pd.DataFrame([[place] + list(d[place].values()) for place in key_set],
columns=["name", "count", "lat", "lon"])
geo = gpd.GeoDataFrame(
    geo,
    geometry=gpd.points_from_xy(geo.lon, geo.lat),
    crs=4326
)

In [ ]:
ax = geo.plot()
# ax.set_xlim(50, 80)
# ax.set_ylim(20, 45)
cx.add_basemap(
    ax,
    crs=geo.crs)
    # zoom=8,
    # source=cx.providers.Stamen.Watercolor
plt.show()









